In [ ]:
!git clone https://github.com/alanyuwenche/PPO-EVO_StockDayTrade

Cloning into 'PPO-EVO_StockDayTrade'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data-1225.zip to data-1225.zip


In [ ]:
ls

gym_wrapper.py  test_N2.pickle  test_N3.pickle  train_N.pickle


In [ ]:
cd PPO-EVO_StockDayTrade/

/content/PPO-EVO_StockDayTrade


In [ ]:
rm -f E*.*

In [ ]:
cd ..

In [ ]:
!tar -cvf data.tar data/
#!gzip data.tar
#!tar zxvf data.tar.gz #解壓縮(須先改名)
#!unzip data.zip

In [ ]:
!unzip data.zip

In [ ]:
!pip install tensorboardX

In [ ]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir=/content/PPO-EVO_StockDayTrade/runs/20221220

In [ ]:
#### 預先載入ppo訓練模型ppoS-113400(已修改於erl_trainer_evo.py), 再以EVO訓練  ############################
#20230102- (test_N>4) and (no_T>1)=> 不理想
!python main.py --env daytradeEnv --popsize 20 --hidden_size 256 --num_test 10 --total_steps 120
#20230101- (test_N>4) and (no_T>1)=> 無結果
#!python main.py --env daytradeEnv --popsize 50 --hidden_size 256 --num_test 10 --total_steps 120
#20221230- (test_N>4) and (no_T>1)=> 不理想
#!python main.py --env daytradeEnv --popsize 30 --hidden_size 256 --num_test 10 --total_steps 120
#20221229- (test_N>2) and (no_T>0)=> Gen598或許可試
#!python main.py --env daytradeEnv --popsize 20 --hidden_size 256 --num_test 10 --total_steps 120

**@@@@@@@@@@@@@@@@@@@@@@@@@  PPO 訓練  @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@**

In [ ]:
#20221225 update_epoch = 16, 50天平均值=> 平均值大表現不會較好; ppoS-113400再用evo訓練(因交易週期短次數多)
#20221222 update_epoch = 16, 20天平均值=> 平均值大表現不會較好; 交易次數太多
import os
import torch
import torch.nn as nn
from torch.distributions import Categorical
import numpy as np
import pickle
import random
from gym_wrapper import *

from collections import Counter
from tensorboardX import SummaryWriter

# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")

class Reporter:
    def __init__(self, report_interval: int = 1):
        self.counter = Counter()
        self.graph_initialized = False
        self.report_interval = report_interval
        self.t = 0
        self.writer = SummaryWriter('runs/20221220')#20221004

    def will_report(self, tag: str) -> bool:
        return self.counter[tag] % (self.report_interval + 1) == 0

    def scalar(self, tag: str, value: float):
        if self.will_report(tag):
            self._scalar(tag, value, self.counter[tag])
        self.counter[tag] += 1

    def graph(self, model, input_to_model):
        if not self.graph_initialized:
            self._graph(model, input_to_model)
            self.graph_initialized = True

    def _scalar(self, tag: str, value: float, step: int):
        self.writer.add_scalar(tag, value, step)

class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

class CategoricalPolicy(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size):
        super(CategoricalPolicy, self).__init__()
        self.action_dim = action_dim

        ######################## Q1 Head ##################
        # Construct Hidden Layer 1 with state
        self.f1 = nn.Linear(state_dim, hidden_size)

        #Hidden Layer 2
        self.f2 = nn.Linear(hidden_size, hidden_size)

        #Value
        self.val = nn.Linear(hidden_size, 1)

        #Advantages
        self.adv = nn.Linear(hidden_size, action_dim)

    def clean_action(self, obs, return_only_action=True):
        ###### Feature ####
        info = torch.relu(self.f1(obs))
        info = torch.relu(self.f2(info))

        val = self.val(info)
        adv = self.adv(info)

        logits = val + adv - adv.mean()

        if return_only_action:
            return logits.argmax(1)

        #return None, None, logits
        return val, adv, logits

    def noisy_action(self, obs, return_only_action=True):
        _, _, logits = self.clean_action(obs, return_only_action=False)

        dist = Categorical(logits=logits)
        action = dist.sample()

        action = action
        action_logprob = dist.log_prob(action)

        if return_only_action:
            return action

        #return action, None, logits
        return action, action_logprob, logits


class PPO:
    def __init__(self, state_dim, action_dim, hidden_size, lr_actor, lr_critic, gamma, K_epochs, eps_clip):
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        self.buffer = RolloutBuffer()
        self.policy = CategoricalPolicy(state_dim, action_dim, hidden_size).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr_actor)
        """
        self.actor = self.policy.adv#????????????????????????
        self.critic = self.policy.val
        self.optimizer = torch.optim.Adam([
                        {'params': self.actor.parameters(), 'lr': lr_actor},
                        {'params': self.critic.parameters(), 'lr': lr_critic}
                    ])
        """
        self.policy_old = CategoricalPolicy(state_dim, action_dim, hidden_size).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        self.MseLoss = nn.MSELoss()
        self.reporter = Reporter()

    def select_action(self, state):

        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logp, _ = self.policy_old.noisy_action(state,return_only_action=False)
        
        self.buffer.states.append(state)
        self.buffer.actions.append(action)
        self.buffer.logprobs.append(action_logp)

        return action.item()

    def evaluate(self, state, action):

        _, _, action_probs = self.policy.noisy_action(state, return_only_action=False)
        dist = Categorical(logits=action_probs)
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        #state_values = self.critic(state)
        state_values, _, _ = self.policy.clean_action(state, return_only_action=False)
        
        return action_logprobs, state_values, dist_entropy

    def update(self):
        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)

        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            # Finding the ratio (pi_theta / pi_theta__old)
            #ratios = torch.exp(logprobs - old_logprobs.detach())#ori
            ratios = torch.exp(logprobs.detach() - old_logprobs)

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            policy_loss = -torch.min(surr1, surr2)
            value_loss = self.MseLoss(state_values, rewards)
            # final loss of clipped objective PPO
            #loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            loss = policy_loss + 0.5*value_loss - 0.01*dist_entropy
            loss = loss.mean()

            # take gradient step
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        self.reporter.scalar('ppo_loss/total', loss)
        self.reporter.scalar('ppo_loss/policy_loss', policy_loss.mean().item())
        self.reporter.scalar('ppo_loss/value_loss', value_loss.mean().item())

        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
print("============================================================================================")

def seed_torch(seed):
        torch.manual_seed(seed)
        if torch.backends.cudnn.enabled:
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = True

def calDN(rew):
  rewL = len(rew)
  noT = 0
  for i in range(rewL):
    if abs(rew[i]) < 5:
      noT += 1
  return noT, rewL



max_ep_len = 300       # max timesteps in one episode
num_epochs = 5000000    
update_epoch = 16      # update policy every n epoch


# Set the seed
seed = 111
np.random.seed(seed)
random.seed(seed)
seed_torch(seed)

################ PPO hyperparameters ################
hidden_dim = 256
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

#####################################################

env_name = "daytradeEnv_PPO"
print("training environment name : " + env_name)
env = GymWrapper(env_name)
num_inputs = env.state_dim
num_actions = env.action_dim

############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")
print("max timesteps per episode : ", max_ep_len)
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(num_inputs, num_actions, hidden_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip)

print("============================================================================================")

os.makedirs('./data/', exist_ok=True)
ret = []
for i_episode in range(num_epochs):    
    state = env.reset()
    current_ep_reward = 0

    for t in range(max_ep_len):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state_next, reward, done, _ = env.step(action)
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        current_ep_reward += reward

        state = state_next
        if done:
            ret.append(current_ep_reward)
            ppo_agent.reporter.scalar('Reward/single_R', current_ep_reward)
            if i_episode > 5 and i_episode % update_epoch == 0:
            #if i_episode % update_epoch == 0:
                ppo_agent.update()
            break

    if i_episode > 5 and i_episode % 50 == 0:
      noTrade, totTrade = calDN(ret)     
      avgR = np.array(ret).mean()
      ppo_agent.reporter.scalar('Reward/avg_R', avgR)
      if avgR > 200:
        f = open("./data/logfile.txt","a")
        f.write("i_episode: %d\t" % i_episode)
        f.write("avg_reward: %d\t" % avgR)
        f.write("交易次數: %d\t" % (totTrade-noTrade))
        f.write("總天數: %d\t" % totTrade)
        f.write("\n")
        f.close()
        fileN = './data/ppoS-'+str(i_episode)+'.pth'
        torch.save(ppo_agent.policy.state_dict(),fileN)
      print("i_episode: ", i_episode, " avg_reward: ",avgR," 交易次數: ",(totTrade-noTrade),"/",totTrade)
      #print("i_episode: ", i_episode, " avg_reward: ",avgR," 交易次數: ",noTrade,"/",totTrade)
      ppo_agent.reporter.scalar('ppo_loss/noTrade', noTrade)
      del ret[:] 



In [ ]:
policy_net._modules['adv'].weight.detach().numpy().shape

(3, 256)

**###########  測試結果  ##############################**

In [ ]:
#測試模型- train_N, test_N, test_N2
import pickle
import numpy as np
import random
import gym

def standardScale_S(aa):
    K = 3.29 #99.9%
    #K = 1.96 #95%
    meanV = 0.00088012709079
    stdV = 0.0104573489
    x_std = (aa-meanV)/(K*stdV)
    return x_std, meanV, stdV 

def standardScale_P(aa):
    K = 3.29 #99.9%
    #K = 1.96 #95%
    meanV = 11136.855753
    stdV = 1094.2235127
    x_std = (aa-meanV)/(K*stdV)
    return x_std, meanV, stdV   

with open('/content/PPO-EVO_StockDayTrade/train_N.pickle','rb') as f:
#with open('test_N2.pickle','rb') as f:
    data = pickle.load(file = f)
l = 284 #13:29=> the last one
dataN = len(data)
ss = np.zeros((l,dataN))
pp = np.zeros((l,dataN))
symBe = ''
base = 0

for i in range(dataN):
    sym = (data[i][1])[3:4] #J, K, L
    if sym != symBe:

        #for s in range(l-1):
        for s in range(l):
            #ss[s,i] = (data[i][0].iloc[s,1]/data[i][0].iloc[0,1]-1.)
            ss[s,i] = np.log(data[i][0].iloc[s,1])-np.log(data[i][0].iloc[0,1])
            pp[s,i] = data[i][0].iloc[s,1]
        symBe = sym
        base = data[i][0].iloc[l-1,1]

    else:
        #for s in range(l-1):
        for s in range(l):
            #ss[s,i] = (data[i][0].iloc[s,1]/base-1.)
            ss[s,i] = np.log(data[i][0].iloc[s,1])-np.log(base)
            pp[s,i] = data[i][0].iloc[s,1]
        symBe = sym
        base = data[i][0].iloc[l-1,1]            

ss, _, _ = standardScale_S(ss)#如果只是測試training data沒變, 但testing data會有差
pp, _, _ = standardScale_P(pp)

class GymWrapper(gym.Env):

  def __init__(self, env_name, frameskip=None):
      self.state_dim = 6
      self.action_dim = 3
      self.pIndex = ss
      self.price_std = pp
      self.price = data
      self.commission = 47
      self.position = np.array([0.])
      self.inventory = []
      self.d = 0 # day
      self.t = 0 # time
      self.st = 0 # tradin time
      self.done = False

  def is_discrete(self, env):
        try:
            k = env.action_space.n
            return True
        except:
            return False

  def getStateTv(self):
      aa = self.pIndex[self.t-4:self.t , self.d]
      pri_s = np.array([self.price_std[self.t-1, self.d]])
      aa = np.concatenate((aa, pri_s, self.position), axis=0)
      return aa

  def _take_action(self, action):
      reward = 0
      if action == 1:
          if int(self.position[0]) == 0:
              self.position = np.array([1.])
              self.inventory.append(self.price[self.d][0].iloc[self.t-1,1])
              self.st = self.t # tradin time

          if int(self.position[0]) == -1:
              sold_price = self.inventory.pop(0)
              reward = 50*(sold_price - self.price[self.d][0].iloc[self.t-1,1])-2*self.commission
              self.done = True
              self.position = np.array([0.])
      elif action == 2:
          if int(self.position[0]) == 0:
              self.position = np.array([-1.])
              self.inventory.append(self.price[self.d][0].iloc[self.t-1,1])
              self.st = self.t # tradin time

          if int(self.position[0]) == 1:
              bought_price = self.inventory.pop(0)
              reward = 50*(self.price[self.d][0].iloc[self.t-1,1] - bought_price)-2*self.commission
              self.done = True
              self.position = np.array([0.])
       
      return reward, self.position

  def step(self, action):
      reward, self.position = self._take_action(action)
      self.t += 1
      #observation = self.getState()
      observation = self.getStateTv()
      if self.t == 284:
          self.done = True
          if len(self.inventory) > 0:
              if int(self.position[0]) == 1:
                  bought_price = self.inventory.pop(0)
                  reward = 50*(self.price[self.d][0].iloc[self.t-1,1] - bought_price)-2*self.commission
                  #observation[self.state_dim+1] = np.array([0.])
                  observation[5] = np.array([0.])#20220509

              elif int(self.position[0]) == -1:
                  sold_price = self.inventory.pop(0)
                  reward = 50*(sold_price - self.price[self.d][0].iloc[self.t-1,1])-2*self.commission
                  #observation[self.state_dim+1] = np.array([0.])
                  observation[5] = np.array([0.])#20220509
      info = {'env.d':self.d, 'env.t':self.t, 'reward':reward, 'tradein t':self.st}
      #print('observation  ',observation)
      #print('self.done  ',self.done)
      return observation, reward, self.done, info

  def reset(self):
      self.position = np.array([0.])
      self.inventory = []
      dth = random.randint(0, self.pIndex.shape[1]-1)
      self.d = dth # day
      self.t = 4 # time
      self.st = 4 # tradin time
      self.done = False

      return self.getStateTv() 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CategoricalPolicy(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size):
        super(CategoricalPolicy, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.f1 = nn.Linear(state_dim, hidden_size)
        self.f2 = nn.Linear(hidden_size, hidden_size)
        self.val = nn.Linear(hidden_size, 1)
        #Advantages
        self.adv = nn.Linear(hidden_size, action_dim)

    def forward(self, x):
        x = torch.from_numpy(x).float().unsqueeze(0)
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        val = self.val(x)
        adv = self.adv(x)
        logits = val + adv - adv.mean()

        return logits.argmax(1).item()

env = GymWrapper("daytradeEnv")
num_inputs = env.state_dim
num_actions = env.action_dim 
num_hidden = 256

policy_net = CategoricalPolicy(num_inputs, num_actions, num_hidden)
state_dict = torch.load('/content/PPO-EVO_StockDayTrade/data/ppoS-113400.pth')
#state_dict = torch.load('/content/Evolutionary-Reinforcement-Learning/data/Gen-2542.pth',map_location=torch.device('cpu')) # GPU用
policy_net.load_state_dict(state_dict)
"""
np.savetxt("f1_weight.csv",policy_net._modules['f1'].weight.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("f1_bias.csv",policy_net._modules['f1'].bias.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("f2_weight.csv",policy_net._modules['f2'].weight.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("f2_bias.csv",policy_net._modules['f2'].bias.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("val_weight.csv",policy_net._modules['val'].weight.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("val_bias.csv",policy_net._modules['val'].bias.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("adv_weight.csv",policy_net._modules['adv'].weight.detach().numpy() ,delimiter=",",fmt='%.6f')
np.savetxt("adv_bias.csv",policy_net._modules['adv'].bias.detach().numpy() ,delimiter=",",fmt='%.6f')
"""
test_epochs = 50
rewards = []
for i_episode in range(test_epochs):
    state = env.reset()
    current_ep_reward = 0
    #env.d = 6 # 同時變更 同時變更 同時變更 同時變更
    #state = env.getStateTv() # 同時變更 同時變更 同時變更 同時變更
    for t in range(300):
        #print('sssssssssss state:',state)
        action = policy_net(state)
        #print('env.d: ',env.d,' env.t: ',env.t,' action: ',action)#!!!!實際驗證盈虧時是用t-1的價格
        state, reward, done, _ = env.step(action)
 
        current_ep_reward += reward

        if done:
            print('i_episode: ',i_episode,' current_ep_reward: ',current_ep_reward,' env.d: ',env.d)
            rewards.append(current_ep_reward)
            break

avg_reward = np.array(rewards)
x = np.where(avg_reward == 0)
avg_R = np.delete(avg_reward, x)
#print('Average reward: {:.2f}'.format(np.mean(avg_R)))
print('Average reward: ', np.mean(avg_R),' numTrade: ',avg_R.shape[0])